In [402]:
#import the essential libraries
import pandas as pd
import numpy as np
import math
import scipy.stats

In [249]:
#Sample 1 given data
sm1 = 2
n1 = 12
sigma1 = 0
s1 = 4.84

In [251]:
#Sample 2 given data
sm2 = 74
n2 = 9
sigma2 = 0
s2 = 9.81

In [253]:
sample_1,sample_2 = [sm1,n1,sigma1,s1],[sm2,n2,sigma2,s2]

In [386]:
def two_sample_hypothesis_testing(
    sample_1, sample_2, alpha=0.95,
    independent=True, tail='two',
    test_type='mean', equal_variances=False
):
    """
    Perform hypothesis testing for two samples (mean or standard deviation).

    Parameters:
        sample_1, sample_2: Tuples of (mean, sample_size, population_std, sample_std)
        alpha: Confidence level (default 0.95)
        independent: If True, samples are independent; otherwise, paired
        tail: 'two', 'left', 'right' or 'two-tailed', 'left-tailed', 'right-tailed'
        test_type: 'mean' or 'standard_deviation'
        equal_variances: Use pooled variance if True

    Returns:
        p_value (rounded), significance_level
    """
    significance = 1 - alpha
    
    # Unpack the sample tuples
    sample_mean_1, sample_size_1, sigma_1, s1 = sample_1
    sample_mean_2, sample_size_2, sigma_2, s2 = sample_2

    # -----------------------------
    # Case 1: Hypothesis Test for Means
    # -----------------------------
    if test_type == 'mean':
        
        # ---- Z-test for known population variances (independent samples)
        if independent and sigma_1 != 0 and sigma_2 != 0:
            z_score = (sample_mean_1 - sample_mean_2) / math.sqrt(
                (sigma_1 ** 2 / sample_size_1) + (sigma_2 ** 2 / sample_size_2)
            )
            print(f"The z_score is {z_score:.4f}")

            if tail == 'two':
                p_value = scipy.stats.norm.sf(abs(z_score)) * 2
            elif tail == 'left':
                p_value = scipy.stats.norm.cdf(z_score)
            elif tail == 'right':
                p_value = scipy.stats.norm.sf(z_score)
            else:
                raise ValueError("Invalid tail argument. Use 'two', 'left', or 'right'.")

        # ---- T-test with pooled variance (equal variances)
        elif independent and equal_variances:
            df = sample_size_1 + sample_size_2 - 2
            sp_2 = (
                ((s1 ** 2) * (sample_size_1 - 1)) +
                ((s2 ** 2) * (sample_size_2 - 1))
            ) / df
            t_score = (sample_mean_1 - sample_mean_2) / math.sqrt(
                sp_2 * ((1 / sample_size_1) + (1 / sample_size_2))
            )
            print(f"The t_score is {t_score:.4f}")

            if tail == 'two':
                p_value = scipy.stats.t.sf(abs(t_score), df) * 2
            elif tail == 'left':
                p_value = scipy.stats.t.cdf(t_score, df)
            elif tail == 'right':
                p_value = scipy.stats.t.sf(t_score, df)
            else:
                raise ValueError("Invalid tail argument. Use 'two', 'left', or 'right'.")

        # ---- Welch’s T-test (unequal variances, independent samples)
        elif independent and not equal_variances:
            numerator = (s1 ** 2 / sample_size_1) + (s2 ** 2 / sample_size_2)
            denominator = ((s1 ** 2 / sample_size_1) ** 2) / (sample_size_1 - 1) + \
                          ((s2 ** 2 / sample_size_2) ** 2) / (sample_size_2 - 1)
            df = numerator ** 2 / denominator

            t_score = (sample_mean_1 - sample_mean_2) / math.sqrt(numerator)
            print(f"Welch's df: {df:.2f}")
            print(f"The t_score is {t_score:.4f}")

            if tail == 'two-tailed':
                p_value = scipy.stats.t.sf(abs(t_score), df) * 2
            elif tail == 'left-tailed':
                p_value = scipy.stats.t.cdf(t_score, df)
            elif tail == 'right-tailed':
                p_value = scipy.stats.t.sf(t_score, df)
            else:
                raise ValueError("Invalid tail argument. Use 'two-tailed', 'left-tailed', or 'right-tailed'.")

        # ---- Paired T-test
        elif not independent and test_type == 'mean':
            df = sample_size_1 - 1
            t_stat = sample_mean_1 / (s1 / math.sqrt(sample_size_1))
            print(f'The t_statistic (paired test) is {t_stat:.4f}')

            if tail == 'two-tailed':
                p_value = 2 * (1 - scipy.stats.t.cdf(abs(t_stat), df))
            elif tail == 'right-tailed':
                p_value = 1 - scipy.stats.t.cdf(t_stat, df)
            elif tail == 'left-tailed':
                p_value = scipy.stats.t.cdf(t_stat, df)
            else:
                raise ValueError("Invalid tail argument. Use 'two-tailed', 'left-tailed', or 'right-tailed'.")

    # -----------------------------
    # Case 2: Hypothesis Test for Standard Deviations (F-test)
    # -----------------------------
    elif test_type == 'standard_deviation':
        dof_num = sample_size_1 - 1
        dof_den = sample_size_2 - 1
        f_stat = (s1 ** 2) / (s2 ** 2)
        print(f"The f_statistic is {f_stat:.4f}")

        if tail == 'right':
            p_value = 1 - scipy.stats.f.cdf(f_stat, dof_num, dof_den)
        elif tail == 'left':
            p_value = scipy.stats.f.cdf(f_stat, dof_num, dof_den)
        elif tail == 'two-tailed':
            if f_stat > 1:
                p_value = 2 * (1 - scipy.stats.f.cdf(f_stat, dof_num, dof_den))
            else:
                p_value = 2 * scipy.stats.f.cdf(f_stat, dof_num, dof_den)
        else:
            raise ValueError("Invalid tail argument. Use 'two-tailed', 'left', or 'right'.")

    else:
        raise ValueError("Invalid test_type. Choose 'mean' or 'standard_deviation'.")

    # -----------------------------
    # Hypothesis Test Conclusion
    # -----------------------------
    if p_value < significance:
        print("✅ Reject the Null Hypothesis")
    else:
        print("❌ Failed to reject the Null Hypothesis")

    return round(p_value, 5), round(significance, 2)

In [388]:
#two_sample_hypothesis_testing(sample_1,sample_2,alpha = 0.05,independent=False,tail = 'right')

In [390]:
mean_diff = 4.5
s1 = 6.845
n1 = 8

In [392]:
sample_1 = [mean_diff,n1,0,s1]

In [394]:
#two_sample_hypothesis_testing(sample_1,sample_2,alpha = 0.05,independent=False,tail = 'left')

In [396]:
#two_sample_hypothesis_testing(sample_1,sample_2,alpha = 0.05,tail = 'right',test_type = 'standard_deviation')

In [398]:
#two_sample_hypothesis_testing(sample_1,sample_2,alpha = 0.02,tail = 'left',test_type = 'standard_deviation')

In [400]:
two_sample_hypothesis_testing(sample_1,sample_2,alpha = 0.05,tail = 'right-tailed',test_type = 'mean',independent = False)

The t_statistic (paired test) is 1.8594
✅ Reject the Null Hypothesis


(0.05265, 0.95)